In [30]:
# detail data per team
# change score_hold to get different subset of the data
# 5 = 25%, 10 = 50%, 16 = 75%, 75 = 100%
import numpy as np
import pandas as pd

reg = pd.read_csv("data/RegularSeasonDetailedResults.csv")
reg['score_diff'] = abs(reg.Wscore - reg.Lscore)

score_hold = 20
data = reg[reg.score_diff<=score_hold]

## 把输赢队的技术统计分开
df1 = data[['Season','Daynum','Wteam','Wscore','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']].copy()
df2 = data[['Season','Daynum','Lteam','Lscore','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']].copy()

dfw=df1.groupby(['Wteam']).mean()
dfl=df2.groupby(['Lteam']).mean()
dfw.head()

dfa = pd.concat([dfw, dfl], axis=1)

dfa = dfa.dropna()

dfa['score'] = dfa['Wscore'] + dfa['Lscore'];
dfa['fgm'] = dfa['Wfgm'] + dfa['Lfgm'];
dfa['fga'] = dfa['Wfga'] + dfa['Lfga'];
dfa['fgm3'] = dfa['Wfgm3'] + dfa['Lfgm3'];
dfa['fga3'] = dfa['Wfga3'] + dfa['Lfga3'];
dfa['ftm'] = dfa['Wftm'] + dfa['Lftm'];
dfa['fta'] = dfa['Wfta'] + dfa['Lfta'];
dfa['or'] = dfa['Wor'] + dfa['Lor'];
dfa['dr'] = dfa['Wdr'] + dfa['Ldr'];
dfa['ast'] = dfa['Wast'] + dfa['Last'];
dfa['to'] = dfa['Wto'] + dfa['Lto'];
dfa['stl'] = dfa['Wstl'] + dfa['Lstl'];
dfa['blk'] = dfa['Wblk'] + dfa['Lblk'];
dfa['pf'] = dfa['Wpf'] + dfa['Lpf'];

data0 = dfa[[-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

dfm=data0/2;
dfm = dfm.reset_index()

tour = pd.read_csv("data/TourneyCompactResults.csv")
tour = tour.rename(columns={"Wteam":"team1", "Lteam":"team2"})
# add a column of game results
tour["result"] = np.repeat(1,2050)
for i in range(2050):
    if tour.team1[i] > tour.team2[i]:       
        team1 = tour.team2[i]
        team2 = tour.team1[i]
        tour.set_value(i, "team1", team1)
        tour.set_value(i, "team2", team2)
        tour.set_value(i, "result", 0)
        
data = tour[[0,2,4,8]]

data = pd.merge(data, dfm, how='left', left_on=["team1"], right_on=["index"])
data = pd.merge(data, dfm, how='left', left_on=["team2"], right_on=["index"])

data = data.dropna()
data0 = data.drop(data.columns[[4,-15]], axis=1)

target = data0['result']
df = data0.drop(data.columns[[0,1,2,3]], axis=1)

In [31]:
# finally we are ready to make some prediction
# import packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.model_selection import cross_val_score

x_train, x_test, y_train, y_test = train_test_split(df, target, random_state=0)

log = LogisticRegression().fit(x_train, y_train)
print(log.score(x_test,y_test));

pred = log.predict(x_test)
confusion = confusion_matrix(y_test,pred)
print(confusion);
print(precision_score(y_test,pred));
print(recall_score(y_test,pred));

logreg = LogisticRegression()
scores = cross_val_score(logreg, df, target, cv=5)
print(scores)

0.656920077973
[[158 114]
 [ 62 179]]
0.610921501706
0.742738589212
[ 0.62530414  0.6902439   0.65609756  0.64146341  0.66992665]


In [ ]:
dfm.to_csv("detail_per_team_all.csv")